In [ ]:
import pandas as pd

data = pd.read_csv("/Users/jeevakumar/Desktop/DS_Lab/data/project_data/UK_Farmer_Tweets.csv")

In [ ]:
data.keys()

In [ ]:
senti_data = data[['Message', 'Sentiment']]

In [ ]:
senti_data.describe()

In [ ]:
senti_data.describe()

In [ ]:
senti_data.info()

In [ ]:
senti_data[senti_data.duplicated(["Message"],keep=False)]

In [ ]:
#senti_data['Sentiment'] = senti_data['Sentiment'].astype('category')

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = ''.join((z for z in text if not z.isdigit()))
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return lemmatized_tokens

senti_data['tokenized_text'] = senti_data['Message'].apply(preprocess_text)

In [ ]:
# from nltk.tokenize import TweetTokenizer
# tt = TweetTokenizer()
# senti_data.Message = senti_data.Message.apply(tt.tokenize)
senti_data['tokenized_text'] 

In [ ]:
senti_data.head()

In [ ]:
#Baseline model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Preprocess the data
senti_data['Sentiment'] = senti_data['Sentiment'].map({'POSITIVE': 2, 'NEUTRAL': 1, 'NEGATIVE': 0})

senti_data.dropna(inplace=True)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(senti_data['Message'], senti_data['Sentiment'], test_size=0.2)

# Extract features from the tweets using bag-of-words representation
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Train the model using Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Test the model on the testing set and evaluate its performance
y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def find_aspects_tweet(doc):
    noun_adj_pairs = {}
    doc = nlp(doc)
    for chunk in doc.noun_chunks:
        adj = set()
        noun = ""
        for tok in chunk:
            if tok.pos_ == "NOUN":
                noun = tok.text
            if tok.pos_ == "ADJ":
                adj.add(tok.text.lower())
        if noun:
            noun_adj_pairs.update({noun.lower():adj})
    return noun_adj_pairs

senti_data['aspects'] = senti_data['Message'].apply(find_aspects_tweet)


In [ ]:
aspects = {}
# finding combining all aspects into a single dictionary
for aspect in senti_data['aspects']:
    for key, value in aspect.items():
        if key not in aspects and len(value) > 0:
            aspects[key] = set()
        elif len(value) > 0:
            aspects[key].update(value)

In [ ]:
# finding common words associated with different aspects
dep_aspects = {}
count = 0 
for key1, value1 in aspects.items():
    for key2, value2 in aspects.items():
        if key1 == key2: # continue if they are the same set
            continue
        keys = sorted([key1, key2])
        combined_key = "{}__{}".format(keys[0], keys[1])
        if combined_key in dep_aspects: # If the pair of aspects are already considered, ignore them
            continue
        inter = value1.intersection(value2)
        if len(inter) > 0: # the sets are indepent with eachother
            dep_aspects[combined_key] = inter

# remove common words from aspect sets making them disjoint
for keys, values in dep_aspects.items():
    aspect1, aspect2 = keys.split("__")
    aspects[aspect1] = aspects[aspect1] - values
    aspects[aspect2] = aspects[aspect2] - values